<a href="https://colab.research.google.com/github/JimKing100/airbnb-app-4/blob/master/Datascience/model/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install category_encoders==2.0.0

In [0]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor


import category_encoders as ce
from joblib import dump

import pandas_profiling as pp

In [0]:
# Load the raw data
data_jan = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-jan19-edited.csv')
data_feb = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-feb19-edited.csv')
data_mar = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-mar19-edited.csv')
data_apr = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-apr19-edited.csv')
data_may = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-may19-edited.csv')
data_jun = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-jun19-edited.csv')
data_jul = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-jul19-edited.csv')
data_aug = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-aug19-edited.csv')
data_sep = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-sep19-edited.csv')
data_oct = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-oct19-edited.csv')
data_n19 = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-nov19-edited.csv')
data_n18 = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-nov18-edited.csv')
data_d18 = pd.read_csv('https://raw.githubusercontent.com/JimKing100/airbnb-app-4/master/Datascience/data/listings-dec18-edited.csv')


In [0]:
df = pd.concat([data_jan, data_feb, data_mar, data_apr, data_may, data_jun, data_jul, data_aug, data_sep, data_oct, data_n19, data_n18, data_d18],
               ignore_index=True)

In [5]:
print(df.shape)
df.head()

(151352, 65)


,id,last_scraped,experiences_offered,host_id,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,market,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,1078,1/15/19,none,4635658,1/8/13,NaN,NaN,t,University of Texas,3.0,3.0,NaN,78705,NaN,Austin,TX,Austin,30.301231,-97.736736,t,Guesthouse,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,85,575.0,1999.0,NaN,35.0,2,125,1,365,today,t,0,0,0,0,1/15/19,208,9/13/08,7/14/17,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{""Texas State""}",t,f,flexible,f,f,3,1.65
1,2265,1/14/19,none,2466,8/23/08,100%,NaN,t,East Downtown,3.0,3.0,East Downtown,78702,NaN,Austin,TX,Austin,30.277500,-97.713975,f,House,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,225,NaN,NaN,500.0,100.0,4,30,2,183,5 days ago,t,12,26,35,157,1/14/19,23,3/17/09,9/16/18,93.0,9.0,10.0,9.0,10.0,8.0,9.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,f,3,0.19
2,5245,1/14/19,none,2466,8/23/08,100%,NaN,t,East Downtown,3.0,3.0,East Downtown,78702,NaN,Austin,TX,Austin,30.275774,-97.713793,t,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,125,NaN,NaN,500.0,75.0,2,25,2,7,6 weeks ago,t,15,26,30,30,1/14/19,9,3/19/09,3/14/18,91.0,10.0,8.0,10.0,9.0,10.0,9.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,f,3,0.08
3,5456,1/14/19,none,8028,2/16/09,100%,NaN,t,East Downtown,1.0,1.0,East Downtown,78702,NaN,Austin,TX,Austin,30.261116,-97.734483,t,Guesthouse,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets live...",NaN,95,NaN,NaN,100.0,NaN,2,45,2,90,1 week ago,t,15,33,57,318,1/14/19,465,3/8/09,12/16/18,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,t,1,3.88
4,5769,1/15/19,none,8186,2/19/09,100%,NaN,t,SW Williamson Co.,1.0,1.0,SW Williamson Co.,78729,NaN,Austin,TX,Austin,30.455964,-97.783702,t,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,40,160.0,NaN,NaN,NaN,2,0,1,14,2 weeks ago,t,20,49,69,69,1/15/19,238,4/10/10,10/20/18,98.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,"{""Texas State""}",f,f,moderate,t,t,1,2.23


In [0]:
def convert_date(scrape_date, m_or_y):
    temp_date = scrape_date
    mdy = temp_date.split('/')
    month = mdy[0]
    year = mdy[2]
    if m_or_y == 'm':
      return int(month)
    else:
      return int(year)

In [7]:
df['month'] = df.apply(lambda x: convert_date(x['last_scraped'], 'm'), axis =1)
df['year'] = df.apply(lambda x: convert_date(x['last_scraped'], 'y'), axis =1)
df.head()

,id,last_scraped,experiences_offered,host_id,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,market,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,month,year
0,1078,1/15/19,none,4635658,1/8/13,NaN,NaN,t,University of Texas,3.0,3.0,NaN,78705,NaN,Austin,TX,Austin,30.301231,-97.736736,t,Guesthouse,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,85,575.0,1999.0,NaN,35.0,2,125,1,365,today,t,0,0,0,0,1/15/19,208,9/13/08,7/14/17,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{""Texas State""}",t,f,flexible,f,f,3,1.65,1,19
1,2265,1/14/19,none,2466,8/23/08,100%,NaN,t,East Downtown,3.0,3.0,East Downtown,78702,NaN,Austin,TX,Austin,30.277500,-97.713975,f,House,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,225,NaN,NaN,500.0,100.0,4,30,2,183,5 days ago,t,12,26,35,157,1/14/19,23,3/17/09,9/16/18,93.0,9.0,10.0,9.0,10.0,8.0,9.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,f,3,0.19,1,19
2,5245,1/14/19,none,2466,8/23/08,100%,NaN,t,East Downtown,3.0,3.0,East Downtown,78702,NaN,Austin,TX,Austin,30.275774,-97.713793,t,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,125,NaN,NaN,500.0,75.0,2,25,2,7,6 weeks ago,t,15,26,30,30,1/14/19,9,3/19/09,3/14/18,91.0,10.0,8.0,10.0,9.0,10.0,9.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,f,3,0.08,1,19
3,5456,1/14/19,none,8028,2/16/09,100%,NaN,t,East Downtown,1.0,1.0,East Downtown,78702,NaN,Austin,TX,Austin,30.261116,-97.734483,t,Guesthouse,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Pets live...",NaN,95,NaN,NaN,100.0,NaN,2,45,2,90,1 week ago,t,15,33,57,318,1/14/19,465,3/8/09,12/16/18,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{""Texas State""}",f,f,strict_14_with_grace_period,f,t,1,3.88,1,19
4,5769,1/15/19,none,8186,2/19/09,100%,NaN,t,SW Williamson Co.,1.0,1.0,SW Williamson Co.,78729,NaN,Austin,TX,Austin,30.455964,-97.783702,t,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,40,160.0,NaN,NaN,NaN,2,0,1,14,2 weeks ago,t,20,49,69,69,1/15/19,238,4/10/10,10/20/18,98.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,"{""Texas State""}",f,f,moderate,t,t,1,2.23,1,19


In [8]:
model_df = df[['month', 'year', 'neighbourhood', 'accommodates', 'bedrooms', 'bathrooms', 'beds', 'price']]
model_df.head()

,month,year,neighbourhood,accommodates,bedrooms,bathrooms,beds,price
0,1,19,NaN,2,1.0,1.0,1.0,85
1,1,19,East Downtown,4,2.0,2.0,2.0,225
2,1,19,East Downtown,2,1.0,1.0,1.0,125
3,1,19,East Downtown,3,1.0,1.0,2.0,95
4,1,19,SW Williamson Co.,2,1.0,1.0,1.0,40


In [9]:
model_df.describe()

,month,year,accommodates,bedrooms,bathrooms,beds,price
count,151352.000000,151352.000000,151352.000000,151293.000000,151082.000000,151165.000000,151352.000000
mean,6.819758,18.845843,4.638095,1.791986,1.543850,2.498773,340.529540
std,3.515242,0.361101,3.131535,1.266074,0.881236,2.450820,895.389008
min,1.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,19.000000,2.000000,1.000000,1.000000,1.000000,80.000000
50%,7.000000,19.000000,4.000000,1.000000,1.000000,2.000000,145.000000
75%,10.000000,19.000000,6.000000,2.000000,2.000000,3.000000,300.000000
max,12.000000,19.000000,33.000000,23.000000,18.500000,132.000000,24999.000000


In [10]:
pp.ProfileReport(model_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,8
Number of observations,151352
Total Missing (%),1.1%
Total size in memory,9.2 MiB
Average record size in memory,64.0 B
Numeric,6
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [11]:
model_df = model_df[(model_df['accommodates'] < 8) &
                    (model_df['bedrooms'] < 6) &
                    (model_df['bathrooms'] < 6) &
                    (model_df['beds'] < 8) &
                    (model_df['price'] < 500) &
                    (pd.notnull(model_df['neighbourhood']))]

print(model_df.shape)
model_df.head()

(107477, 8)


,month,year,neighbourhood,accommodates,bedrooms,bathrooms,beds,price
1,1,19,East Downtown,4,2.0,2.0,2.0,225
2,1,19,East Downtown,2,1.0,1.0,1.0,125
3,1,19,East Downtown,3,1.0,1.0,2.0,95
4,1,19,SW Williamson Co.,2,1.0,1.0,1.0,40
6,1,19,Zilker,3,1.0,1.0,2.0,129


In [12]:
val = model_df[(model_df['month'] == 10) & (model_df['year'] == 19)]
val = val.reset_index(drop=True)
print(val.shape)
val.head()

(8119, 8)


,month,year,neighbourhood,accommodates,bedrooms,bathrooms,beds,price
0,10,19,East Downtown,4,2.0,2.0,2.0,225
1,10,19,East Downtown,2,1.0,1.0,2.0,100
2,10,19,East Downtown,3,1.0,1.0,2.0,95
3,10,19,SW Williamson Co.,2,1.0,1.0,1.0,40
4,10,19,Zilker,3,1.0,1.0,2.0,129


In [13]:
test = model_df[(model_df['month'] == 11) & (model_df['year'] == 19)]
test = test.reset_index(drop=True)
print(test.shape)
test.head()

(7934, 8)


,month,year,neighbourhood,accommodates,bedrooms,bathrooms,beds,price
0,11,19,East Downtown,4,2.0,2.0,2.0,225
1,11,19,East Downtown,2,1.0,1.0,2.0,100
2,11,19,East Downtown,3,1.0,1.0,2.0,95
3,11,19,SW Williamson Co.,2,1.0,1.0,1.0,40
4,11,19,Zilker,3,1.0,1.0,2.0,129


In [14]:
train = model_df.drop(model_df[(model_df['month'] == 10) & (model_df['year'] == 19)].index)
print(train.shape)
train.head()

(99358, 8)


,month,year,neighbourhood,accommodates,bedrooms,bathrooms,beds,price
1,1,19,East Downtown,4,2.0,2.0,2.0,225
2,1,19,East Downtown,2,1.0,1.0,1.0,125
3,1,19,East Downtown,3,1.0,1.0,2.0,95
4,1,19,SW Williamson Co.,2,1.0,1.0,1.0,40
6,1,19,Zilker,3,1.0,1.0,2.0,129


In [15]:
train = train.drop(train[(train['month'] == 11) & (train['year'] == 19)].index)
train = train.reset_index(drop=True)
print(train.shape)
train.head()

(91424, 8)


,month,year,neighbourhood,accommodates,bedrooms,bathrooms,beds,price
0,1,19,East Downtown,4,2.0,2.0,2.0,225
1,1,19,East Downtown,2,1.0,1.0,1.0,125
2,1,19,East Downtown,3,1.0,1.0,2.0,95
3,1,19,SW Williamson Co.,2,1.0,1.0,1.0,40
4,1,19,Zilker,3,1.0,1.0,2.0,129


In [16]:
print(train.shape, val.shape, test.shape)

(91424, 8) (8119, 8) (7934, 8)


In [22]:
# Encode and fit a Random Forest model
target = 'price'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestClassifier(n_estimators=50, 
                           random_state=42,
                           max_features='auto',
                           n_jobs=-1,
                           verbose = 1)
)

pipeline.fit(X_train, y_train)
#dump(pipeline, 'pipeline.joblib')
y_pred = pipeline.predict(X_val)

print('Validation Accuracy', accuracy_score(y_val, y_pred))

# Print metrics for validation 
# val_mse = mean_squared_error(y_val, y_pred)
# val_rmse = np.sqrt(val_mse)
# val_mae = mean_absolute_error(y_val, y_pred)
# val_r2 = r2_score(y_val, y_pred)
# print('Validation Mean Absolute Error:', val_mae)
# print('Validation R^2:', val_r2)
# print('\n')

# ty_pred = pipeline.predict(X_test)

# Print metrics for test
# test_mse = mean_squared_error(y_test, ty_pred)
# test_rmse = np.sqrt(test_mse)
# test_mae = mean_absolute_error(y_test, ty_pred)
# test_r2 = r2_score(y_test, ty_pred)
# print('Test Mean Absolute Error:', test_mae)
# print('Test R^2:', test_r2)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


Validation Accuracy 0.3226998398817588


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.5s finished
